**Expectation Maximization** - алгоритм построения параметрической вероятностной модели, основанный на итеративном подборе ее параметров

## Мотивация

Для начала вспомним классическую задачу MLE
$$\Theta_{opt} = argmax_{\Theta} [ logP(X|\Theta)]$$

Здесь нам нужно найти параметры модели, лучше всего объясняющие наблюдаемые данные

Но с расчетом возникает сложность, если $P(X|\Theta)$ зависит не только от параметров модели, но и от скрытых (латентных) переменных: $P(X|\Theta) = P(X|Z, \Theta)$

В этом случае нам нужно взвешивать (формулой полной веротяности) по возможным значениям Z $$P(X|\Theta) = \sum_{Z} P(X|Z,\Theta) P (Z|\Theta)$$

## Алгоритм (математически)
1. Считаем $$Q(\Theta|\Theta_{current}) = E_{Z|X,\Theta_{current}} logP(X|Z,\Theta)$$
2. Максимизируем $$\Theta_{new} = argmax_{\Theta} Q(\Theta)$$
Повторяем до сходимости $$\Theta_0, \Theta_1 \cdots \Theta_n \rightarrow \Theta_{opt}$$

-----

Важно пояснить нотацию:
$Q = E_{Z|X,\Theta_{current}} logP(X|Z,\Theta)$ - это взвешенное среднее, то есть 
- $\sum P(Z|X,\Theta) logP(X|Z,\Theta)$ для дискретных латентных переменных


- $\int P(Z|X) logP(X|Z,\Theta) d \Theta$ для непрерывных



Также важно отметить, что $\Theta_{current}$ - это фиксированное значение параметра (результат предыдущей итерации алгоритма), а $\Theta$ - то, по чему на втором шаге мы оптимизируем. Отсюда нотация $Q(\Theta|\Theta_{current})$


## Алгоритм ("на пальцах"):
1. Фиксируем параметры модели $\Theta_{current}$ и ищем наиболее вероятный набор Z
2. Фиксируем значения $Z$ и подбираем наиболее веротяные значения параметра $\Theta$

Легче всего алгоритм иллюстрируется на примере кластеризации k-means или гауссовых смесей

## Приложения
- K-means
- Gaussian Mix Models
- Topic Modeling
- Hidden Markov Models

# Теоретическая обоснованность алгоритма

Функцию правдоподобия можно оценить снизу:


$$L(\theta) = logP(X|\theta) \ge E_{Z|X} logP(X,Z) + H(X) = ELBO$$ 

ELBO = Evidence Lower Bound

Схема работы алгоритма показана на картинке ниже:
- красная линия - искомая функция правдоподобия (нам не видна)
- синяя - ELBO для первой оценки
- зеленая - ELBO для второй оценки
<img src="img/em.jpeg" width=400>

# Иллюстрация на примере K-means

### Дано 
Некоторая выборка на плоскости, состоящая из N точек

<img src="img/kmeans_0.png">
    
Принадлежность точки классу определяется латентной переменной $Z=(z_1 \cdots z_N)$. Считаем, что в ней представлены данные 3 классов и $z \in \{'red','blue','green'\}$. То есть вектор **Z** - это раскраска множества точек выборки в 3 цвета. 
    
Параметры $\Theta = (\theta_1, \theta_2, \theta_3)$ характеризуют центры распределения каждого из классов $\{x|x \in X_1\},\{x|x \in X_2\},\{x|x \in X_3\}$. Раскраска в k-means однозначно опредлеяется параметрами $\Theta$.

### Задача
Определить оптимальный набор $\Theta$, который наиболее правдоподобно описывает данные. Раскраска (Z) тогда будет рассчитана автоматически и должна выглядеть примерно так:

<img src = "img/kmeans_3.png">

### Алгоритм

#### 0. Initialization

Берем некий начальный параметр $\Theta_0$. Значения латентной переменной **Z** зависят от $\Theta$: чем ближе точка к центру кластера, тем больше веротяность этого кластера $P(Z|\Theta)$, 

<img src="img/kmeans_1.png">


#### 1 Expectation

$E_{Z|X,\Theta}logP(X,Z) = \sum P(Z|X,\Theta_0)logP(X|Z,\Theta)$

- $Z|X\Theta$

    Смотрим условное распределение вероятности латентных переменных Z при фиксированном $\Theta$: $$P(Z|X,\Theta)$$

    Для начала посчитаем $P(z_i|x_i,\Theta)$ для отдельной точки - она могла быть сгенерирована из любого из трех классов, поэтому ее приндлежность - апостериорная веротяность, которая рассчитывается по теореме Байеса, как взвешенная сумма правдоподобий каждого класса.

    Далее $P(Z|X,\Theta)$ считается как произведение $P(z_i|x_i,\Theta)$

    Соджержательно это ответ на вопрос - Как распределены значения латентных переменных (разделение на классы) при заданном $\Theta$
    

- $log P(X|Z,\Theta)$

    Смотрим правдоподобие выборки $X$ при данных $Z$ и $\Theta$. То есть при заданной раскраске $Z$ и заданных распределениях $\Theta$, какова веротяность наблюдать данную выборку.

- $E(\Theta)$

    

Далее, так как k-means это у нас пример hard-clustering, то мы дальше мы работаем не с распределением, а берем наиболее вероятный набор значений. Он легко определяется без особой математики - всё  что ближе к центру кластера X, относится к кластеру X.

<img src="img/kmeans_2.png">


#### 2. Maximization 

Теперь фиксируем Z, который мы нашли на предыдущем шаге, и корректируем параметр $\Theta$. 

Выбираем на плоскости такие центры распределения, которые лучше всего описывают текущие значения Z. Лучше всего = правдоподобие $P(X|Z,\Theta)$ наибольшее.\

# Иллюстрация на примере GMM (gaussian mix model)

Он же soft-clustering